In [0]:
read_json = spark.read.json('dbfs:/databricks-datasets/samples/people/people.json')
display(read_json)

In [0]:
#display(dbutils.fs.ls('/databricks-datasets/samples'))
display(dbutils.fs.ls('dbfs:/databricks-datasets/samples/newsgroups/20_newsgroups'))

In [0]:
1. schema enforcement in json?
2. flatten json : arrays and struct
3. create delta table
4. dedup data ways and best one way and why
5. Deduplication in Spark Streaming
6. join > broadcast > hint join stragey
7. merge logic 
8. scd 1/2/3
9. schema registery 
10. schema enforcement
11. mode: permissive etc
12. read data from file using spark/sql which and why
13. how to copy/insert/migrate data from one table to another OR file to table OR file to file, tradeoffs etc: when to create table and when not to


In [0]:
iot_json_df = spark.read.json('/databricks-datasets/iot/iot_devices.json')

In [0]:
display(iot_json_df)

In [0]:
#generate sample json dtaa file
import json

# 1. Define a Python dict that maps to the desired JSON structure
data = {
    "id": 1,                                # simple datatype (int)
    "name": "Alice",                        # simple datatype (string)
    "is_active": True,                      # simple datatype (boolean)
    "score": 95.5,                          # simple datatype (float)

    # array of simple datatypes
    "tags": ["premium", "beta_user", "india"],

    # struct (object) with simple fields
    "profile": {
        "email": "alice@example.com",
        "age": 30,
        "country": "IN"
    },

    # nested struct (struct inside struct)
    "address": {
        "line1": "123 Main Street",
        "city": "New Delhi",
        "postal_code": "110001",
        "geo": {                            # nested struct
            "lat": 28.6139,
            "lon": 77.2090
        }
    },

    # array of structs
    "orders": [
        {
            "order_id": "ORD-1001",
            "amount": 1200.50,
            "items": [
                {"sku": "SKU-1", "qty": 1},
                {"sku": "SKU-2", "qty": 2}
            ]
        },
        {
            "order_id": "ORD-1002",
            "amount": 800.00,
            "items": [
                {"sku": "SKU-3", "qty": 3}
            ]
        }
    ]
}

# 2. Convert to JSON string (pretty-printed for readability)
json_str = json.dumps(data, indent=2)

# 3. Write to DBFS using dbutils.fs.put
#    Target path *in DBFS* (visible under /dbfs/ in the driver)
target_path = "/Volumes/hk/samplejson/jsonfiles/nested_example.json"

dbutils.fs.put(target_path, json_str, overwrite=True)

print(f"Wrote JSON to {target_path}")


In [0]:
dbutils.fs.ls('/Volumes/hk/samplejson/jsonfiles/')

In [0]:
df=spark.read.json('/Volumes/hk/samplejson/jsonfiles/nested_example.json')
display(df )


### CSV – Raw Ingestion (Bronze)
**Exercise 1: Basic CSV Load + Schema Control**

Scenario: You receive daily CSV drops from a vendor in ADLS. Schema sometimes breaks (extra column, wrong data type).

Dataset (create mock):
- customer_id,name,age,signup_date
- 1,Alice,29,2024-01-01
- 2,Bob,,2024-01-03
- 3,Charlie,thirty,2024-01-05

Tasks:
- Load CSV with: 
- header
- explicit schema
- Capture corrupt records
- Write to Bronze Delta

In [0]:
data = [
    (1,"Alice",29,"2024-01-01"), 
    (2,"Bob", None,"2024-01-03"),
    (3,"Charlie","thirty","2024-01-05")
    #,(None,4,"darcy",87,"2024-01-08")
]

columns = ["cutomer_id", "name", "age", "signup_date"]

df = spark.createDataFrame(data, columns)
df.write.mode("overwrite").option("header", "true").csv("/Volumes/hk/sample/samplecsv/customerData")

In [0]:
type(data)

In [0]:
import pandas as pd

data = {
    "customer_id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie","Darcy"],
    "age": ["29", None, "thirty",20],
    "signup_date": ["2024-01-01", "2024-01-03", "2024-01-05", "0"]

}

df = pd.DataFrame(data)

output_path = "/Volumes/hk/sample/samplecsv/customer_data_Test0Date.csv"
df.to_csv(output_path, index=False)

print(f"CSV written to {output_path}")


In [0]:
df = spark.read.csv('/Volumes/hk/sample/samplecsv/customer_data_Test0Date.csv', header=True, schema="customer_id INT, name STRING, age STRING, signup_date DATE")
display(df)

##CSV

In [0]:
import pandas as pd

data = {
    "customer_id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie","Darcy"],
    "age": ["29", None, "thirty",20],
    "signup_date": ["2024-01-01", "2024-01-03", "2024-01-05", "0"]
}

df = pd.DataFrame(data)

output_path = "/Volumes/hk/sample/samplecsv/customer_data_Test0Date.csv"
df.to_csv(output_path, index=False)

print(f"CSV written to {output_path}")

In [0]:
import pandas as pd

data = {
    "cust_id": [1,2,3,4,5],
    "age":[23,32,41,19, 21],
    "name":["alice","bob","charlie","daren","Eugene"],
    "sign_up_date":['01-01-2018', '01-03-2024', '01-05-2024', '01-08-2024','2021-00-08']
}

# data = {
#     "cust_id": [1,2,3,4],
#     "age":[23,32,41,19],
#     "name":["alice","bob","charlie","daren"],
#     "sign_up_date":['01-01-2018', '01-03-2024', '01-05-2024', '01-08-2024']
# }
_sample_csv = pd.DataFrame(data)

_sample_csv.to_csv("/Volumes/hk/sample/samplecsv/customer_data.csv", index = False, mode='w')

In [0]:
_sample_csv.head()

Spark’s CSV reader expects dates to match its default pattern yyyy−MM−dd. When you declare a DateType column and do not specify a date format.

​The sign_up_date values are coming in as null because Spark cannot parse the string dates in your CSV into a DateType using its default date pattern, so it silently sets them to null in PERMISSIVE mode.

If the data is in dd-MM-yyyy format (for example, 01-01-2018), which does not match the default, so parsing fails and Spark cannot convert these strings into DateType. And hence, spark will read those record as NULL
 ​

With mode="PERMISSIVE", Spark does not throw an error; instead, it sets the unparseable fields to null and, if configured, stores the original row in the _corrupt_record column.​


In [0]:
#OPTION 1: LOAD sign_up_date as StringType and later convert to date using to_date() function

from pyspark.sql.functions import col, to_date

csv_schema_2 = StructType([
    StructField("cust_id", IntegerType(), False),
    StructField("age", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("sign_up_date", StringType(), False),
    StructField("_corrupt_record", StringType(), True)
])

cust_csv_raw_op2  = spark.read.option("header","true")\
    .schema(csv_schema_2)\
    .option("mode","PERMISSIVE")\
    .option("columnNameOfCorruptRecord", "_corrupt_record")\
    .csv("/Volumes/hk/sample/samplecsv/customer_data.csv")


cust_csv = cust_csv_raw_op2.select(
    "cust_id",
    "age",
    "name",
    to_date("sign_up_date", "dd-MM-yyyy").alias("sign_up_date")
)
display(cust_csv)

In [0]:
# Load CSV with:
# header
# explicit schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
csv_schema = StructType([
    StructField("cust_id", IntegerType(), False),
    StructField("age", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("sign_up_date", DateType(), False),
    StructField("_corrupt_record", StringType(), True)
])



#OPTION2: use option("dateFormat", "dd-MM-yyyy") during spark.read.csv()—it's significantly faster as parsing happens natively in the optimized CSV reader without extra transformations. Preferred for high-volume CSV data

cust_csv_raw_op1  = spark.read.option("header","true")\
    .schema(csv_schema)\
    .option("mode","PERMISSIVE")\
    .option("columnNameOfCorruptRecord", "_corrupt_record")\
    .option("dateFormat", "dd-MM-yyyy")\
    .csv("/Volumes/hk/sample/samplecsv/customer_data.csv")
display(cust_csv_raw_op1)



In [0]:
cust_csv_raw_op1.filter(col("_corrupt_record").isNull()).write.mode("overwrite").option("header", "true").csv("/Volumes/hk/sample/delta/validRecords")

In [0]:
cust_csv_raw_op1.filter(col("_corrupt_record").isNotNull()).write.mode("overwrite").option("header", "true").csv("/Volumes/hk/sample/delta/inValidRecords")

In [0]:
whata re possible data converions and casting?

In [0]:
f.show()

In [0]:
Flatten JSON

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/"))

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/travel_recommendations_realtime/raw_travel_data/fs-demo_destination-availability_logs/json"))

In [0]:
js = spark.read.json("dbfs:/databricks-datasets/travel_recommendations_realtime/raw_travel_data/fs-demo_destination-availability_logs/json")
display(js)

In [0]:
js = spark.read.json(
    "dbfs:/databricks-datasets/travel_recommendations_realtime/raw_travel_data/fs-demo_destination-availability_logs/json"
)
display(js)


In [0]:
# Small files are handled by increasing partitions’ output size and compacting. This includes writing data with an appropriate number of output partitions (or using 
# coalesce
# /
# repartition
#  before writes), and running periodic compaction jobs that read many small files and rewrite them into fewer large ones, which improves metadata load and scan performance on data lakes



# Example: batch write to Parquet with controlled number of output files
target_num_partitions = 64  # tune based on data size & cluster

df_clean = (
    input_df
        .filter("event_date >= '2025-01-01'")
        .select("user_id", "event_time", "metric1", "metric2")
)

# If you're increasing partitions or want a balanced layout, use repartition (shuffle = true)
df_to_write = df_clean.repartition(target_num_partitions)

# If you're only decreasing partitions after heavy shuffles, coalesce is cheaper:
# df_to_write = df_clean.coalesce(target_num_partitions)

(
    df_to_write
        .write
        .mode("append")
        .format("parquet")
        .save("abfss://raw@account.dfs.core.windows.net/app/events")
)


In [0]:
# Example: partitioned Parquet with controlled file count per batch
daily_df = (
    input_df
        .withColumn("event_date", F.to_date("event_time"))
        .filter("event_date = '2025-12-25'")
)

# Choose partitions so that per-partition file size is close to your target (e.g. 256–1024 MB)
daily_df = daily_df.repartition(64, "event_date")

(
    daily_df
        .write
        .mode("append")
        .partitionBy("event_date")
        .format("parquet")
        .save("abfss://raw@account.dfs.core.windows.net/app/events")
)


In [0]:
from pyspark.sql import functions as F

source_path = "abfss://raw@account.dfs.core.windows.net/app/events"
target_path = source_path                   # in-place compaction
num_target_files = 64                       # tune based on total volume

# Optional: compact only specific partition(s)
partition_to_compact = "event_date = '2025-12-25'"

df = (
    spark.read
         .format("parquet")
         .load(source_path)
         .where(partition_to_compact)
)

(
    df.repartition(num_target_files)
      .write
      .mode("overwrite")       # overwrites that partition
      .format("parquet")
      .save(target_path)
)


In [0]:
from delta.tables import DeltaTable

delta_path = "abfss://silver@account.dfs.core.windows.net/app/events_delta"
num_target_files = 32

(
    spark.read.format("delta").load(delta_path)
         .where("event_date = '2025-12-25'")
         .repartition(num_target_files)
         .write
         .option("dataChange", "false")  # important for Delta compaction
         .format("delta")
         .mode("overwrite")
         .save(delta_path)
)


In [0]:
%sql
-- compact the whole table
OPTIMIZE catalog.schema.events_delta;

-- or compact only a partition
OPTIMIZE catalog.schema.events_delta
WHERE event_date = '2025-12-25';


In [0]:
from delta.tables import DeltaTable

delta_tbl = DeltaTable.forName(spark, "catalog.schema.events_delta")

# Full-table compaction
delta_tbl.optimize().executeCompaction()

# Partition-specific compaction (Databricks API)
delta_tbl.optimize().where("event_date = '2025-12-25'").executeCompaction()


In [0]:
%sql
-- At table level
ALTER TABLE catalog.schema.events_delta
SET TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);


In [0]:
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")


read csv file with schema evolution/register 

In [0]:
from pyspark.sql import functions as F

# v1 CSV (old schema)
df_v1 = (
    spark.read
         .format("csv")
         .option("header", "true")
         .schema("id string, name string")     # old schema
         .load("/path/to/csv/v1")
)

# v2 CSV (new schema with extra column)
df_v2 = (
    spark.read
         .format("csv")
         .option("header", "true")
         .schema("id string, name string, age int")  # new schema
         .load("/path/to/csv/v2")
)

# Normalized target schema (superset)
target_cols = ["id", "name", "age"]

# Add missing columns with nulls
df_v1_norm = df_v1.withColumn("age", F.lit(None).cast("int"))[target_cols]
df_v2_norm = df_v2.select(target_cols)

# Union all versions
df_all = df_v1_norm.unionByName(df_v2_norm)

# Register or write
df_all.createOrReplaceTempView("people_csv")  # register
# or better: write to Delta with schema evolution
(
    df_all.write
        .format("delta")
        .option("mergeSchema", "true")
        .mode("append")
        .save("/mnt/delta/people")
)
